<a href="https://colab.research.google.com/github/AlexGitta/DAID/blob/main/prototype3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MuJoCo imitation prototype

For this prototype I decided I wanted to focus on creating an interactive and easy to use human imitation program in MuJoCo. Therefore, I did not attempt to implement custom pose imitation, instead I used the keyframes included in the humanoid model by DeepMind. My future plan is to create a pipeline where users can perform poses or motions that the agent will then try to imitate. I decided to use Colab for this early prototype because I understand you are busy and I wanted to make it easy for you to run and view, but I am currently working on a local implementation and I do not plan to use Colab for the final product. I understand we are encouraged to submit a Git repository to show our prototyping process, and I will submit this with the real prototype.

Vocabulary
  - CoM - Centre of Mass
  - DoF - Degree of Freedom
  - LQR - Linear Quadratic Regulator

## Installs and Imports (Setup)

In [17]:
# Install packages using pip
!pip install mujoco
!pip install -q mediapy

In [18]:
# set up GPU rendering in colab
from google.colab import files
import distutils.util
import os
import subprocess
if subprocess.run('nvidia-smi').returncode:
  raise RuntimeError(
      'Cannot communicate with GPU. '
      'Make sure you are using a GPU Colab runtime. '
     'Go to the Runtime menu and select Choose runtime type.')
else:
  print('GPU connected.')

NVIDIA_ICD_CONFIG_PATH = '/usr/share/glvnd/egl_vendor.d/10_nvidia.json'
if not os.path.exists(NVIDIA_ICD_CONFIG_PATH):
  with open(NVIDIA_ICD_CONFIG_PATH, 'w') as f:
    f.write("""{
    "file_format_version" : "1.0.0",
    "ICD" : {
        "library_path" : "libEGL_nvidia.so.0"
    }
}
""")

%env MUJOCO_GL=egl


GPU connected.
env: MUJOCO_GL=egl


In [19]:
# Import packages
try:
  import mujoco
  import mediapy as media
  import numpy as np
  from typing import Callable, Optional, Union, List
  import scipy.linalg
  import matplotlib.pyplot as plt
  from IPython.display import clear_output
except Exception as e:
   raise e from RuntimeError(
      'Something went wrong during installation. Check the shell output above ')

print('Installed packages')

Installed packages


In [20]:
# Clone humanoid from github
!git clone https://github.com/google-deepmind/mujoco
with open('mujoco/model/humanoid/humanoid.xml', 'r') as f:
  humanoid = f.read()
  model = mujoco.MjModel.from_xml_string(humanoid)
  data = mujoco.MjData(model)

Cloning into 'mujoco'...
remote: Enumerating objects: 26288, done.
remote: Counting objects: 100% (447/447), done.
remote: Compressing objects: 100% (208/208), done.
error: 1856 bytes of body are still expected
fetch-pack: unexpected disconnect while reading sideband packet
fatal: early EOF
fatal: fetch-pack: invalid index-pack output


FileNotFoundError: [Errno 2] No such file or directory: 'mujoco/model/humanoid/humanoid.xml'

## Set up and render humanoid


Here I load the humanoid into the model and data variables, then render and create a video where the model is instantiated into the specified pose. Feel free to change the specified pose, and see how the model falls/ragdolls due to no joint actuators being used.

In [ ]:
renderer = mujoco.Renderer(model)

DURATION  = 3   # seconds
FRAMERATE = 60  # Hz

# 0 for squat pose, 1 for balancing pose
mujoco.mj_resetDataKeyframe(model, data, 1)

frames = []
while data.time < DURATION:
  mujoco.mj_step(model, data)

  # render and save frames
  if len(frames) < data.time * FRAMERATE:
    renderer.update_scene(data)
    pixels = renderer.render()
    frames.append(pixels)

# use frames to create and display video
media.show_video(frames, fps=FRAMERATE)

This browser does not support the video tag.


The problem we are now presented with is operating the joint actuators at a minimum cost. We have a desired acceleration (zero, as we are attempting to balance), and we have to calculate the required forces on each joint to achieve this. To do this, we will use SciPy to create a Linear-Quadratic regulator, as well as inverse dynamics to work out the optimal control and minimise the cost function.

## Inverse dynamics
We will set the desired position to our pose, set the desired acceleration to zero and then use the inverse dynamics function in MuJoCo to take the desired acceleration and compute the forces required to create it. Here we print the forces found by the inverse dynamics for the one leg balance pose:

In [15]:
mujoco.mj_resetDataKeyframe(model, data, 1)
mujoco.mj_forward(model, data)  # step forward once
data.qacc = 0  # desired acceleration is zero
mujoco.mj_inverse(model, data)
print(data.qfrc_inverse)

[ 0.00000000e+00  0.00000000e+00  2.75878566e+02 -3.31855341e+01
  4.99547913e+00 -6.68771278e+00 -4.30520866e+00  3.69324103e+00
 -1.54511509e+01 -1.09062996e+01  4.11835550e-01 -1.61300544e+00
 -9.79255144e+00 -2.31182930e+00 -3.66106442e-01 -5.91255024e+00
 -4.17112597e-01 -1.91408228e+00  5.75890779e+00  2.66518759e+00
 -2.02066315e-01 -5.75454992e+00  9.94329230e-01  1.14109244e+00
 -1.98740242e+00  3.82112740e+00  1.15105404e+00]


The largest force is clearly the 3rd in the list, the vertical aspect of the root joint. This is true for both the one leg and the squat pose (Try it yourself! Just change the one for a zero). This is not natural, as we as humans cannot just will our bodies upward to the sky, as if the rapture has come. To fix this, we have to offset the humanoid a little bit from the ground.

### Finding the optimal offset from the ground


This function tries 2001 different offsets and calculates the optimal one where the vertical forces are minimal.

In [7]:
def findOffset(squatorbalance):

    height_offsets = np.linspace(-0.001, 0.001, 2001) # create array of 2001 potential optimal offsets
    vertical_forces = []
    for offset in height_offsets: # loop through array
      mujoco.mj_resetDataKeyframe(model, data, squatorbalance)
      mujoco.mj_forward(model, data) # step forward once
      data.qacc = 0
      data.qpos[2] += offset  # offset the height
      mujoco.mj_inverse(model, data)   # use inverse dynamics to calculate required forces
      vertical_forces.append(data.qfrc_inverse[2])  # add the force to the list

    # find the offset at which the force is smallest.
    idx = np.argmin(np.abs(vertical_forces))
    best_offset = height_offsets[idx]
    return best_offset


print(findOffset(1))

NameError: name 'np' is not defined

Using this function in our original force calculation results in alot cleaner forces.

In [8]:
def calculateForces(squatorbalance):
  mujoco.mj_resetDataKeyframe(model, data, squatorbalance)
  mujoco.mj_forward(model, data)
  data.qacc = 0
  data.qpos[2] += findOffset(squatorbalance)
  qpos0 = data.qpos.copy()
  mujoco.mj_inverse(model, data)
  qfrc0 = data.qfrc_inverse.copy()
  return qfrc0, qpos0

calculateForces(1)

NameError: name 'mujoco' is not defined

As you can hopefully see, these forces are a lot more regular. Now all we have to do is use MuJoCo's genius to calculate the actuator values that will create these forces, and then save them into a

## Calculating our first actuator values

In [6]:
def calculateActuatorValues(squatorbalance):
  qfrc0, qpos0 = calculateForces(squatorbalance)
  actuator_moment = np.zeros((model.nu, model.nv))
  mujoco.mju_sparse2dense(
      actuator_moment,
      data.actuator_moment.reshape(-1),
      data.moment_rownnz,
      data.moment_rowadr,
      data.moment_colind.reshape(-1),
  )
  ctrl0 = np.atleast_2d(qfrc0) @ np.linalg.pinv(actuator_moment)
  ctrl0 = ctrl0.flatten()
  data.ctrl = ctrl0
  return ctrl0, qpos0

print(calculateActuatorValues(1))

NameError: name 'calculateForces' is not defined

Now we can simulate the agent with their new knowledge of the first actuator values.

In [20]:
renderer = mujoco.Renderer(model)
ctrl0, qpos0 = calculateActuatorValues(1) # change to choose squat or one leg balance

DURATION  = 3   # seconds
FRAMERATE = 60  # Hz
mujoco.mj_resetData(model, data)
data.qpos = qpos0
data.ctrl = ctrl0

frames = []
while data.time < DURATION:
  mujoco.mj_step(model, data)

      # render and save frames
  if len(frames) < data.time * FRAMERATE:
    renderer.update_scene(data)
    pixels = renderer.render()
    frames.append(pixels)

media.show_video(frames, fps=FRAMERATE)

This browser does not support the video tag.


Here we can see for both pose attempts, the humanoid still falls, but it does attempt to stabilize itself and fight against gravity for a moment. Next, we need to allow it to feedback to itself, so it can update the actuator forces as the simulation happens. First, we set up a cost for balancing that will aim to keep the centre of mass over the foot (or feet if squatting). We will use kinematic jacobians and use MuJoCo to compute them because it is GENIUS.

## Creating cost functions

In [2]:
def balancingCost(squatorbalance):
    ctrl0, qpos0 = calculateActuatorValues(squatorbalance)
    nu = model.nu  # alias for the number of actuators
    R = np.eye(nu)
    nv = model.nv  # shortcut for the degrees of freedom

    # Get the Jacobian for the root body (torso) CoM
    mujoco.mj_resetData(model, data)
    data.qpos = qpos0
    mujoco.mj_forward(model, data)
    jac_com = np.zeros((3, nv))
    mujoco.mj_jacSubtreeCom(model, data, jac_com, model.body('torso').id)

    if(squatorbalance == 1):
      jac_foot = np.zeros((3, nv))
      mujoco.mj_jacBodyCom(model, data, jac_foot, None, model.body('foot_left').id)

    if(squatorbalance == 0):
      jac_foot_left = np.zeros((3, nv))
      mujoco.mj_jacBodyCom(model, data, jac_foot_left, None, model.body('foot_left').id)
      jac_foot_right = np.zeros((3, nv))
      mujoco.mj_jacBodyCom(model, data, jac_foot_right, None, model.body('foot_right').id)
      jac_foot = (jac_foot_left + jac_foot_right)/2


    jac_diff = jac_com - jac_foot
    Qbalance = jac_diff.T @ jac_diff
    return Qbalance

print(balancingCost(1))

NameError: name 'calculateActuatorValues' is not defined

We need another cost function to ensure the humanoid maintains the pose, or atleast as close to it as possible. The most important joints, e.g. the legs and abdominals, need the strictest cost function, while the less crucial joints, such as the arms, are less strict, allowing the humanoid to move itself to remain balanced, as a human does.

In [3]:
def poseCost(squatorbalance):
    nv = model.nv  # shortcut for the degrees of freedom
    # Get all joint names
    joint_names = [model.joint(i).name for i in range(model.njnt)]

    # Get indices into relevant sets of joints.
    root_dofs = range(6)
    body_dofs = range(6, nv)
    abdomen_dofs = [
        model.joint(name).dofadr[0]
        for name in joint_names
        if 'abdomen' in name
        and not 'z' in name
    ]
    left_leg_dofs = [
        model.joint(name).dofadr[0]
        for name in joint_names
        if 'left' in name
        and ('hip' in name or 'knee' in name or 'ankle' in name)
        and not 'z' in name
    ]
    if squatorbalance == 0:
        right_leg_dofs = [
            model.joint(name).dofadr[0]
            for name in joint_names
            if 'right' in name
            and ('hip' in name or 'knee' in name or 'ankle' in name)
            and not 'z' in name
        ]
        balance_dofs = abdomen_dofs + left_leg_dofs + right_leg_dofs
    else:
        balance_dofs = abdomen_dofs + left_leg_dofs

    other_dofs = np.setdiff1d(body_dofs, balance_dofs)
    return balance_dofs, other_dofs, root_dofs

I had to specify that for the balancing on one leg pose, the weight should be distributed over only one leg. This is a shortcoming of this prototype as it cannot easily be generalised to new poses, which is the end goal of my project. I will focus on how I can generalise and optimise the balanceCost in my next prototype.

It is now time to define the cost for the LQR controller, which the controller will try to minimise and, in effect, achieve stability in their pose.

In [4]:
def constructQ(squatorbalance, balance_cost, important_joint_cost, other_joint_cost):
    nv = model.nv  # shortcut for the degrees of freedom
    # get indices into relevant sets of joints
    balance_dofs, other_dofs, root_dofs = poseCost(squatorbalance)

    Qjoint = np.eye(nv)
    Qjoint[root_dofs, root_dofs] *= 0  # don't penalize root joint, as it is essentially ther CoM, which is already penalised
    Qjoint[balance_dofs, balance_dofs] *= important_joint_cost # multiply balancing joints deviation by cost
    Qjoint[other_dofs, other_dofs] *= other_joint_cost # multiply other joints deviation by correlating cost

    # calculate full cost
    Qpos = balance_cost * balancingCost(squatorbalance) + Qjoint

    # not penalising speed of joint movements, only deviation from pose
    Q = np.block([[Qpos, np.zeros((nv, nv))],
                  [np.zeros((nv, 2*nv))]])
    return Q

Now we have to compute two matrices for the LQR. One (The "A" matrix) for what would naturally happen to the humanoid with no control input or actuators, and the other (The "B" matrix) for what would change in response to control inputs. Together, they allow the algorithm to predict how the humanoid's state will change over time, therefore allowing it to calculate the optimal control actions to keep it upright.

In [5]:
def constructK(squatorbalance):
    nu = model.nu
    nv = model.nv
    ctrl0, qpos0 = calculateActuatorValues(squatorbalance)  # get initial setpoint

    mujoco.mj_resetData(model, data) # assign setpoints
    data.ctrl = ctrl0
    data.qpos = qpos0

    A = np.zeros((2*nv, 2*nv)) # create a matrix with dimensions of the DoF of the humanoid, to represent state transitions
    B = np.zeros((2*nv, nu)) # create another matrix to represent control input
    epsilon = 1e-6
    flg_centered = True
    # use mujoco magic to compute and fill the matrices
    mujoco.mjd_transitionFD(model, data, epsilon, flg_centered, A, B, None, None)#
    return A, B

## Final simulation with customisable variables

Ok, now we are finally done with computation and calculation, let's simulate! This function will take in a few customisable values, which you can change and see how it affects the pose imitation.

In [25]:
def simulate(squatorbalance, balance_cost, important_joint_cost, other_joint_cost, duration, framerate, noise_amount):

    renderer = mujoco.Renderer(model, height = 720, width = 960)

    A, B = constructK(squatorbalance)
    Q = constructQ(squatorbalance, balance_cost, important_joint_cost, other_joint_cost)
    R = np.eye(model.nu) # number of actuators
    ctrl0, qpos0 = calculateActuatorValues(squatorbalance)
    nu = model.nu

    # use scipy to calculate LQR controller
    P = scipy.linalg.solve_discrete_are(A, B, Q, R)

    # calculate the feedback gain matrix, determines how the controller will respond
    K = np.linalg.inv(R + B.T @ P @ B) @ B.T @ P @ A

    # reset data, set initial pose.
    mujoco.mj_resetData(model, data)
    data.qpos = qpos0

    # create array to store difference between current and target position
    dq = np.zeros(model.nv) # model.nv is the DoF of the model

    frames = []
    while data.time < duration:
      # calcualte difference between current and target pose
      mujoco.mj_differentiatePos(model, dq, 1, qpos0, data.qpos)
      dx = np.hstack((dq, data.qvel)).T

      # calculate control signals needed to be sent to actuators to maintain balance
      data.ctrl = ctrl0 - K @ dx

      noise = noise_amount * np.random.randn(nu)
      data.ctrl += noise

      # step the simulation
      mujoco.mj_step(model, data)

      # render
      if len(frames) < data.time * framerate:
        renderer.update_scene(data)
        pixels = renderer.render()
        frames.append(pixels)

    media.show_video(frames, fps=framerate)

Now it's time to see the final product. Rendering takes a few seconds on the Colab GPU, and only a few minutes if using CPU rendering. I have prefilled the variables with values that produce a realistic looking human balancing motion, but please mess around with the values and see what happens! The noise parameter is crucial in making it look realistic and humanlike, as humans can rarely hold a balancing position absolutely perfectly (which the humanoid can do due to its computational power) , and usually have to make small microadjustments. Try and make him fall over!

In [1]:
squatorbalance = 1 # 1 for one leg balance, 0 for squat balance

balance_cost        = 1000  # importance of keeping CoM over foot
important_joint_cost  = 3     # importance of leg/abdomen joint movement
other_joint_cost   = .3    # importance of other joints

duration = 5    # seconds of simulation
framerate = 50    # frames / second

noise_amount = 0.1

simulate(squatorbalance, balance_cost, important_joint_cost, other_joint_cost, duration, framerate, noise_amount)

NameError: name 'simulate' is not defined

# Conclusion
In conclusion, this prototype was more of an exercise in learning about MuJoCo and how to use it than a fully fledged interactive application. I used colab because I wasn't sure what system you were using and this requires no local computational power. For my other experiments I am using local computation as I recently got a fancier laptop so I am able to simulate and render at a decent rate.

## Future

Obviously this prototype is very limited, and is really more of a proof of concept than a minimum viable product. There are clear flaws in the workflow, such as the manual assignment of the balancingCost() function, which would definitely need to be improved if this was to work with unseen poses.
-

I am currently working on a pipeline to take 2D poses from webcam videos or images and convert them to the keyframe format that this humanoid accepts. I have done various experiments with PoseNet and OpenPose, two frameworks for simple pose recognition. I then have to write a conversion script to take these poses and format them.
-

The other, and much less tangible task, is how to make this a more enjoyable interactive experience. At the moment it is just somewhat interesting to look at, and doesn't really engage users, especially those who have little interest in physics simulations / motion imitation. I am in the process of showing this to my peers and having them experiment with it, and will be updating it accordingly.
-

I definitely want to keep the functionality of adding noise or some other way of artifically unbalancing the humanoid. Some other programs I have experimented with, such as MJMPC (https://github.com/google-deepmind/mujoco_mpc), include functionality for dragging limbs of the humanoid and throwing items at them to mess with the algorithm. Live interactivity is important to me, as I think human brains much prefer interactivity where they can see the effect of their change immediately instead of having to wait, so my next prototype will definitely try to incorporate this (Another good reason to leave colab is the lack of live simulation capability).

## Updated timeline

Instead of just diving straight into attempting motion imitation, I have realised that it is probably a better idea to attempt pose imitation first, and then move on to motion if I am happy with the product and have time. Therefore the next thing I will be working on is functionality to create custom poses for the humanoid to replicate. I am now familiar with the format of the keyframes in the DeepMind humanoid (as a product of this prototype) and I will use this knowledge going forward. I know this prototype is not perfect, but I hope you understand this is quite technical and mathsy and it took me a while to wrap my head around it all.


Following your advice, I have spent time thinking about the objectives and aims of my project, and have come up with the following:

## Aims

- To create an interactive system using real-time pose detection and physics simulation, delivering an opportunity to engage with the project both physically (through your body) and virtually (through the simulation GUI)

- Another possible aim could be a competitive aspect - what if users could be tasked with modifying parameters to get the agent to hold their pose for the longest time, or be the most stable?


## Objectives

- Make use of real-time pose detection, achieving at least 80% pose accuracy with a low latency.

- Seamlessly intergrate pose data into MuJoco so it can be tasked with imitation.

- Add user interactivity with parameters such as Risk, various Costs and artificial Noise (discussed above)

- Implement a feedback system for users, I originally thought just the humanoid succesfully mimicing the pose or movement would be enough feedback, but I now think the UX could be enhanced by some other sort of feedback system, perhaps auditory or using data visualisation (Fancy coloured graphs).
